# Preparación de datos.
#### Autor: Sebastián Cifuentes Carrasco
Teniendo una proyección de la tabla gsm_events, donde se extrajeron las columnas:

| id | antenna_id | network_type | date | sim_serial_number |
|----|-----|-----|-----|-----|


Y las tabla de antenas de Santiago, etiquetadas con las comunas a las que pertenecen, y de carriers que otorgan los servicios de antena, haremos un preprocesamiento de datos para trabajar a futuro con estas tablas y dejarlas listas para visualización.




In [1]:
import pandas as pd
import math

In [2]:
#Cargamos el dataset de gsm_events reducido a las columnas que nos importan.
#Header del dataset: id, antenna_id, network_type, date, sim_serial_number
gsm_events = pd.read_csv('gsm_events_mini.csv')

#Cargamos las antenas de Santiago
#Header de antenas: id, cid, lac, lat, lon, carrier_id, comuna
at_stgo = pd.read_csv('antenas_santiago.csv')

#Cargamos los carriers de las antenas
#Header de carriers: id, name, mcc, mnc
carriers = pd.read_csv('carriers.csv')


## Chequeo de valores nulos
Vamos a revisar si las tablas sobre las cuales trabajaremos contienen valores nulos dentro de sus registros. De ser así, habrá que ver un modo de reemplazar estos valores para que no existan problemas a futuro.

In [3]:
# pandasDataFrame -> None
# Chequea si existen datos nulos dentro del dataframe entregado
def checkfornulls(df):
    #Identificamos las columnas donde hay valores null
    null_cols = df.columns[df.isnull().any()]
    #Si es vacío, entonces se informa.
    if null_cols.empty:
        print('No hay columnas con datos nulos')
    else:
        #En caso contrario, contamos los null totales por cada columna y describimos cada columna
        print("Columnas con su respectiva cantidad de registros nulos")
        print(df[null_cols].isnull().sum())
        print()
        for col in null_cols:
            print(df[col].describe())
            print()

In [4]:
#Vemos la información de tipos de la tabla gsm_events
print(gsm_events.info())
#Notar que las fechas son strings. Hay que hacer una conversión a fechas entendibles por Python.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19366931 entries, 0 to 19366930
Data columns (total 5 columns):
id                   int64
antenna_id           float64
network_type         int64
date                 object
sim_serial_number    object
dtypes: float64(1), int64(2), object(2)
memory usage: 738.8+ MB
None


In [5]:
#Revisamos si hay valores null en gsm_events
checkfornulls(gsm_events)

Columnas con su respectiva cantidad de registros nulos
antenna_id           293422
sim_serial_number     39620
dtype: int64

count    1.907351e+07
mean     1.908284e+04
std      1.459131e+04
min      1.000000e+00
25%      6.041000e+03
50%      1.856600e+04
75%      2.723800e+04
max      1.009200e+05
Name: antenna_id, dtype: float64

count                 19327311
unique                     439
top       89560900000040207779
freq                   5767824
Name: sim_serial_number, dtype: object



In [6]:
#Como sabemos que hay valores nulos en la tabla, y son valores de identificación, nos deshacemos de las observaciones
gsm_events.dropna(inplace = True)
#Además, modificamos el tipo de antenna_id para que sea el mismo que en la tabla de antenas
gsm_events['antenna_id'] = gsm_events.antenna_id.astype('int')
#Y chequeamos nuevamente que la limpieza de nulos haya funcionado
checkfornulls(gsm_events)

No hay columnas con datos nulos


In [7]:
#Chequeo de tipos de la tabla de antenas de Santiago.
print(at_stgo.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51487 entries, 0 to 51486
Data columns (total 7 columns):
id            51487 non-null int64
cid           51487 non-null int64
lac           51487 non-null int64
lat           51487 non-null float64
lon           51487 non-null float64
carrier_id    51487 non-null int64
comuna        51487 non-null object
dtypes: float64(2), int64(4), object(1)
memory usage: 2.7+ MB
None


In [8]:
#Revisamos si existen valores null en la tabla de antenas.
checkfornulls(at_stgo)

No hay columnas con datos nulos


In [9]:
#Chequeo de tipos de la tabla carriers.
print(carriers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 4 columns):
id      54 non-null int64
name    54 non-null object
mcc     54 non-null int64
mnc     54 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.8+ KB
None


In [10]:
#Revisamos si existen valores null en la tabla carriers.
checkfornulls(carriers)

No hay columnas con datos nulos


## Preparando el join
Ya teniendo la tabla de eventos limpia de valores nulos, así como las tablas de antenas de Santiago y carriers verificadas, los datos están listos para unirse y dejarlos dispuestos para su análisis. Así, la idea será generar una nueva tabla que contenga la información de cada evento con la información de su respectiva antena y nombre del carrier que otorga el servicio.

## Primera tabla: antenas con sus carriers
La idea es generar una tabla de antenas con el siguiente esquema:

| id | cid | lac | lat | lon | carrier_id | comuna | carrier_name | mcc | mnc |
|----|-----|-----|-----|-----|------------|--------|--------------|-----|-----|

generando una tabla con igual cantidad de observaciones que la de antenas de Santiago (51487 filas).


In [11]:
#Primero, renombramos la columna 'id' de la tabla carrier_id para poder hacer el join.
carriers.rename(columns={'id':'carrier_id'}, inplace=True)
#Y hacemos el join entre las antenas y carriers segun el carrier_id
at_stgo_con_carrier = pd.merge(at_stgo, carriers, on='carrier_id', how='inner')
#Renombramos la columna 'name' para que quede el esquema antes propuesto
at_stgo_con_carrier.rename(columns = {'name': 'carrier_name'}, inplace = True)

In [12]:
print(at_stgo_con_carrier.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51487 entries, 0 to 51486
Data columns (total 10 columns):
id              51487 non-null int64
cid             51487 non-null int64
lac             51487 non-null int64
lat             51487 non-null float64
lon             51487 non-null float64
carrier_id      51487 non-null int64
comuna          51487 non-null object
carrier_name    51487 non-null object
mcc             51487 non-null int64
mnc             51487 non-null int64
dtypes: float64(2), int64(6), object(2)
memory usage: 4.3+ MB
None


Listo!
Ahora falta hacer el join entre la tabla de gsm_events y la nueva tabla de antenas con carrier.
## Segunda tabla: eventos con antenas con carriers
En este caso el esquema que esperamos es el siguiente:

| event_id | antenna_id | network_type | date | sim_serial_number | cid | lac | lat | lon | carrier_id | comuna | carrier_name | mcc | mnc |
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|

Generando una tabla con 4626184 registros (calculado a partir de un INNER JOIN dentro de SQL).


In [13]:
#Primero, cambiamos el nombre la columa 'id' de las antenas con carrier
at_stgo_con_carrier.rename(columns = {'id': 'antenna_id'}, inplace = True)
#Hacemos el join según antenna_id
gsm_events_fulldata = pd.merge(gsm_events, at_stgo_con_carrier, on='antenna_id', how = 'inner')
gsm_events_fulldata.rename(columns = {'id': 'event_id'}, inplace = True)

In [14]:
print(gsm_events_fulldata.info())
#Guardamos la tabla en csv para trabajarla
gsm_events_fulldata.to_csv('gsm_events_fulldata.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4626184 entries, 0 to 4626183
Data columns (total 14 columns):
event_id             int64
antenna_id           int64
network_type         int64
date                 object
sim_serial_number    object
cid                  int64
lac                  int64
lat                  float64
lon                  float64
carrier_id           int64
comuna               object
carrier_name         object
mcc                  int64
mnc                  int64
dtypes: float64(2), int64(8), object(4)
memory usage: 529.4+ MB
None


Listo!
Ya tenemos una tabla de 14 columnas con los eventos GSM de Santiago, georeferenciados y con toda la información relevante para su visualización.